In [49]:
# Import required libraries  
import os  
import json  
import openai  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt
from IPython.display import display, HTML, JSON, Markdown

# Configure environment variables  
load_dotenv() 

True

In [50]:
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY
#---
credential = AzureKeyCredential(key)

print (OPENAI_DEPLOYMENT_ENDPOINT)

https://openailabs303474.openai.azure.com/


In [51]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=500,
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,
                  max_tokens=max_tokens,
                  model_kwargs={"stop": ["<|im_end|>"]}
    ) 
    return llm

In [52]:
# Generate Document Embeddings using OpenAI Ada 002

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(page):
    response = openai.Embedding.create(
        input=page, engine="text-embedding-ada-002")
   
    embeddings = response['data'][0]['embedding']
    return embeddings

In [53]:
#test the model 
llm = init_llm()
llm("hello! How are you today")


'? I hope you are doing well. I am very interested in your project. I have read your project description very carefully. I can make the android app for you. I have developed many android apps for More\n\nHello, I have gone through your project details . I can build your mobile application as per your requirements.I am an experienced mobile app developer with an experience of 5+ years. I’m pretty confident that I will More\n\nHello, I have gone through your project details . I can build your mobile application as per your requirements.I am an experienced mobile app developer with an experience of 5+ years. I’m pretty confident that I will More\n\nHello, I have gone through your project details . I can build your mobile application as per your requirements.I am an experienced mobile app developer with an experience of 5+ years. I’m pretty confident that I will More\n\nHello, I have gone through your project details . I can build your mobile application as per your requirements.I am an ex

In [54]:
template = """ System:
You are assistant helping the company technical support users with their questions about different product features. 
Answer ONLY with the facts listed in the sources below delimited by triple backticks.
If there isn't enough information in the Sources, say you don't know and ask a user to provide more details. 
Do not generate answers that don't use the Sources below. 
If asking a clarifying question to the user would help, ask the question. 

Sources:
```{sources}```
User question is here below delimited by triple backticks
User:
```{question}``` 

You answer is here below:
Answer:
<|im_end|>
"""

In [55]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate


#ConversationBufferMemory is a memory that stores the conversation history
memory = ConversationBufferMemory()
#try to change the verbose to True, to see more details
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)


In [56]:

search_client = SearchClient(service_endpoint, index_name="sk-cogsrch-vector-index", credential=credential)  

In [57]:

def search(question, top_k=3):
    results = search_client.search(  
        search_text=None,
        vector=generate_embeddings(question),  
        top_k=top_k,  
        vector_fields="contentVector",
        select=["title", "content"],
    )  
    
    result_pages = []
    for result in results:
        result_pages.append(result['content'])      

    sources = "\n\n".join([page for page in result_pages])
    return sources    


In [58]:

def ask_openai(sources, question):
    prompt = ChatPromptTemplate.from_template(template=template)
    response = conversation.run (input=prompt.format(sources=sources,
                            question=question))
    return response


In [43]:
question = "what's semnatic kernel?"
sources = search(question, top_k=3)
print (sources)
response = ask_openai(sources, question)
display (Markdown(response))

Tell us about y our PDF experience.
What is Semantic Kernel?
Article •07/11/2023
Semantic K ernel is an open-source SDK that lets you easily combine AI services like
OpenAI , Azure OpenAI , and Hugging F ace  with conventional programming
languages like C# and Python. By doing so, you can create AI apps that combine the
best of both worlds.
During K evin Scott's talk The era of the AI Copilot , he showed how Microsoft powers its
Copilot system  with a stack of AI models and plugins. At the center of this stack is an AI
orchestration layer that allows us to combine AI models and plugins together to create
brand new experiences for users.
Semantic Kernel is at the center of the copilot
stack

Additional learning for Semantic Kernel
Article •07/11/2023
Want to learn more about Semantic K ernel? Check out these in-depth tutorials and
videos. W e will add more content over time from our team and community, so check
back often!
Cook with Semantic Kernel
Learn how to supercharge your problem-

   Semantic Kernel is an open-source SDK that lets you easily combine AI services like OpenAI, Azure OpenAI, and Hugging Face with conventional programming languages like C# and Python. By doing so, you can create AI apps that combine the best of both worlds. During Kevin Scott's talk "The era of the AI Copilot", he showed how Microsoft powers its Copilot system with a stack of AI models and plugins. At the center of this stack is an AI orchestration layer that allows us to combine AI models and plugins together to create brand new experiences for users. Semantic Kernel is at the center of the copilot stack.

In [44]:
question = "which programming languages are supported by semantic kernel?"
sources = search(question, top_k=3)
print (sources)
response = ask_openai(sources, question)
display (Markdown(response))

Supported Semantic Kernel languages
Article •07/18/2023
Semantic K ernel plans on providing support to the following languages:
While the overall architecture of the kernel is consistent across all languages, we made
sure the SDK for each language follows common paradigms and styles in each language
to make it feel native and easy to use.
Today, not all features are available in all languages. The following tables show which
features are available in each language. The 🔄  symbol indicates that the feature is
partially implemented, please see the associated note column for more details. The ❌
symbol indicates that the feature is not yet available in that language; if you would like
to see a feature implemented in a language, please consider contributing to the project
or opening an issue .
Services C# Python JavaNotes
TextGeneration ✅✅✅ Example: T ext-Davinci-003
TextEmbeddings ✅✅✅ Example: T ext-Embeddings-Ada-002
ChatCompletion ✅✅❌ Example: GPT4, Chat-GPT７ Note
Skills are currently be

   Semantic Kernel plans on providing support to the following languages: C#, Python, Java (coming soon). While the overall architecture of the kernel is consistent across all languages, the SDK for each language follows common paradigms and styles in each language to make it feel native and easy to use. Today, not all features are available in all languages. The following tables show which features are available in each language. The 🔄 symbol indicates that the feature is partially implemented, please see the associated note column for more details. The ❌ symbol indicates that the feature is not yet available in that language; if you would like to see a feature implemented in a language, please consider contributing to the project or opening an issue.

In [46]:
question = "Provide steps for using Azure OpenAI model with semantic kernel"
sources = search(question, top_k=3)
print (sources)
response = ask_openai(sources, question)
display (Markdown(response))

Learn how to deploy Semantic Kernel to
Azure as a web app service
Article •05/24/2023
In this how-to guide we will provide steps to deploy Semantic K ernel to Azure as a web
app service. Deploying Semantic K ernel as web service to Azure provides a great
pathway for developers to take advantage of Azure compute and other services such as
Azure Cognitive Services for responsible AI and vectorized databases.
You can use one of the deployment options to deploy based on your use case and
preference.
1. Azure currently limits the number of Azure OpenAI resources per region per
subscription to 3. Azure OpenAI is not available in every region. (R efer to this
availability map ) Bearing this in mind, you might want to use the same Azure
OpenAI instance for multiple deployments of Semantic K ernel to Azure.
2. F1 and D1 App Service SKU's (the Free and Shared ones) are not supported for this
deployment.
3. Ensure you have sufficient permissions to create resources in the target
subscription.
4. 

   You can use an existing Azure OpenAI instance and connect the Semantic Kernel web API to it. To do so, follow the steps in the "Use existing: Azure OpenAI Resources" option in the "Considerations" section of the "Learn how to deploy Semantic Kernel to Azure as a web app service" source.

In [59]:

question = "explain how to deploy Semantic Kernel to Azure as a web app service"
sources = search(question, top_k=3)
print (sources)
response = ask_openai(sources, question)
display (Markdown(response))

Learn how to deploy Semantic Kernel to
Azure as a web app service
Article •05/24/2023
In this how-to guide we will provide steps to deploy Semantic K ernel to Azure as a web
app service. Deploying Semantic K ernel as web service to Azure provides a great
pathway for developers to take advantage of Azure compute and other services such as
Azure Cognitive Services for responsible AI and vectorized databases.
You can use one of the deployment options to deploy based on your use case and
preference.
1. Azure currently limits the number of Azure OpenAI resources per region per
subscription to 3. Azure OpenAI is not available in every region. (R efer to this
availability map ) Bearing this in mind, you might want to use the same Azure
OpenAI instance for multiple deployments of Semantic K ernel to Azure.
2. F1 and D1 App Service SKU's (the Free and Shared ones) are not supported for this
deployment.
3. Ensure you have sufficient permissions to create resources in the target
subscription.
4. 

 To deploy Semantic Kernel to Azure as a web app service, you can use one of the deployment options to deploy based on your use case and preference. You can use an existing Azure OpenAI instance and connect the Semantic Kernel web API to it. After working locally, i.e. you cloned the code from the GitHub repo and have made changes to the code for your needs, you can deploy your changes to Azure as a web application. You can use the standard methods available to deploy an ASP.net web app in order to do so. Alternatively, you can follow the steps below to manually build and upload your customized version of the Semantic Kernel service to Azure. First, at the command line, go to the '../semantic-kernel/samples/apps/copilot-chat-app/webapi' directory and enter the following command: dotnet publish CopilotChatApi.csproj --configuration Release --arch x64 --os win. This will create a directory which contains all the files needed for a deployment. Zip the contents of that directory and store the resulting zip file on cloud storage, e.g. Azure Blob Container. Put its URI in the "Package Uri" field in the web deployment page you access through the "Deploy to Azure" buttons or use its URI as the value for the PackageUri parameter of the deployment scripts found on this page. Your deployment will then use your customized deployment package. That package will be used to create a new Azure web app, which will be configured to run your customized version of the Semantic Kernel service.